# Create Maps

Maps are created by this notebook

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime
import os

In [10]:
outPath = f"output{os.sep}"
dataPath = f"datasets{os.sep}"

In [3]:
def lp(v):
    print(f"[{datetime.now()}] {v}")

lp("Starting...")

[2025-03-27 15:36:47.766045] Starting...


In [5]:
lp(f"outPath:  {outPath}")
lp(f"dataPath:  {dataPath}")

[2025-03-27 15:36:55.514326] outPath:  output/
[2025-03-27 15:36:55.514407] dataPath:  dataset/


In [7]:
lp("Loading 100m lidar data")
data100m = gpd.read_parquet(f"{outPath}AggregateLidarData_100m.parquet")
lp("Loading 10m subset lidar data")
data10m = gpd.read_parquet(f"{outPath}AggregateLidarData_SomeHoods_10m.parquet")

[2025-03-27 15:37:06.838524] Loading 100m lidar data
[2025-03-27 15:37:06.976196] Loading 10m subset lidar data


In [13]:
lp("Loading neighborhoos data")
neighborhoodsDf = gpd.read_file(f"{dataPath}Neighborhoods.geojson")

[2025-03-27 15:38:04.912475] Loading neighborhoos data


In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
neighborhoodsDf.plot(color='pink', ax=ax)
data100m.plot(color='blue', ax=ax)
data10m.plot(color='red',ax=ax)